In [1]:
import turtle

def draw_triangle(vertices, pen):
    """Draws a triangle connecting three given points."""
    pen.penup()
    pen.goto(vertices[0])  
    pen.pendown()
    pen.goto(vertices[1])  
    pen.goto(vertices[2])  
    pen.goto(vertices[0])  

def midpoint(p1, p2):
    """Finds the midpoint between two points."""
    return ((p1[0] + p2[0]) / 2, (p1[1] + p2[1]) / 2)

def sierpinski(vertices, depth, pen):
    """Recursively draws the Sierpinski Triangle."""
    draw_triangle(vertices, pen)  

    if depth > 0:
        # Find midpoints
        mid1 = midpoint(vertices[0], vertices[1])
        mid2 = midpoint(vertices[1], vertices[2])
        mid3 = midpoint(vertices[0], vertices[2])

        # Recursively draw three smaller triangles
        sierpinski([vertices[0], mid1, mid3], depth - 1, pen)
        sierpinski([vertices[1], mid1, mid2], depth - 1, pen)
        sierpinski([vertices[2], mid2, mid3], depth - 1, pen)

# Set up the drawing
screen = turtle.Screen()
screen.bgcolor("white")

pen = turtle.Turtle()
pen.speed(0)  # Max speed for fastest drawing
pen.hideturtle()  # Hide turtle cursor for a cleaner look

# Define the main triangle's points
size = 300
triangle_points = [(-size, -size), (0, size), (size, -size)]

# Start drawing the Sierpinski Triangle
sierpinski(triangle_points, 6, pen)

turtle.done()  # Finish drawing


TclError: invalid command name ".!canvas"

In [ ]:
import turtle

pen = turtle.Turtle()
pen.speed(2)
x1 = (-300, -300)
x2 = (0, 220)
x3 = (300, -300)

def draw_triangle(x1, x2, x3):
    pen.penup()
    pen.goto(x1)
    pen.pendown()
    pen.goto(x2)
    pen.goto(x3)
    pen.goto(x1)
    # draw_triangle(x1, x2, x3)

draw_triangle(x1, x2, x3)

: 

In [4]:
import random

def generateGraph(n, density='sparse'):
    edges = set()
    max_edges = n * (n - 1) // 2
    
    if density == 'sparse':
        m = min(max_edges, n * 2)  # ~2 edges per node
    elif density == 'dense':
        m = min(max_edges, int(max_edges * 0.75))  # 75% of all possible edges
    else:
        raise ValueError("Density must be 'sparse' or 'dense'")
    
    while len(edges) < m:
        u = random.randint(0, n - 1)
        v = random.randint(0, n - 1)
        if u != v:
            edge = (min(u, v), max(u, v))  # avoid duplicates
            if edge not in edges:
                w = random.randint(1, 100)
                edges.add((u, v, w))
    
    return n, list(edges)

def saveGraphToFile(n, edges, filename):
    with open(filename, 'w') as f:
        f.write(f"{n} {len(edges)}\n")
        for u, v, w in edges:
            f.write(f"{u} {v} {w}\n")

# Sizes and types
sizes = [10, 100, 1000]
densities = ['sparse', 'dense']

for size in sizes:
    for density in densities:
        n, edges = generateGraph(size, density)
        filename = f"{size}_{density}Graph.txt"
        saveGraphToFile(n, edges, filename)
        print(f"Generated {filename}")


Generated 10_sparseGraph.txt
Generated 10_denseGraph.txt
Generated 100_sparseGraph.txt
Generated 100_denseGraph.txt
Generated 1000_sparseGraph.txt
Generated 1000_denseGraph.txt


In [14]:
import time
from heapq import heappop, heappush

# ---------- PRIM'S ALGORITHM ----------
def primsAlgorithm(adjList, root=0):
    parentArray = [-1] * len(adjList)
    key = [float('inf')] * len(adjList)
    treeVertex = [False] * len(adjList)
    key[root] = 0
    minHeap = [(key[root], root)]
    while minHeap:
        minKey, u = heappop(minHeap)
        if treeVertex[u]:
            continue
        treeVertex[u] = True
        for v, w in adjList[u]:
            if not treeVertex[v] and key[v] > w:
                key[v] = w
                parentArray[v] = u
                heappush(minHeap, (key[v], v))
    return sum(key)

# ---------- KRUSKAL'S ALGORITHM ----------
def find(parent, i):
    if parent[i] != i:
        parent[i] = find(parent, parent[i])
    return parent[i]

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)
    if rank[xroot] < rank[yroot]:
        parent[xroot] = yroot
    elif rank[xroot] > rank[yroot]:
        parent[yroot] = xroot
    else:
        parent[yroot] = xroot
        rank[xroot] += 1

def kruskalsAlgorithm(n, edgeList):
    edgeList.sort(key=lambda x: x[2])
    parent = [i for i in range(n)]
    rank = [0] * n
    mst_weight = 0
    count = 0

    for u, v, w in edgeList:
        x = find(parent, u)
        y = find(parent, v)
        if x != y:
            mst_weight += w
            union(parent, rank, x, y)
            count += 1
            if count == n - 1:
                break
    return mst_weight

# ---------- GRAPH LOADING ----------
def loadGraph(filename):
    with open(filename, 'r') as f:
        n, m = map(int, f.readline().split())
        adjList = {i: [] for i in range(n)}
        edgeList = []
        for _ in range(m):
            u, v, w = map(int, f.readline().split())
            adjList[u].append((v, w))
            adjList[v].append((u, w))
            edgeList.append((u, v, w))
    return n, adjList, edgeList

# ---------- RUN + TIME ----------
def runAndTimeAlgorithms(filename):
    n, adjList, edgeList = loadGraph(filename)

    start = time.time()
    prim_weight = primsAlgorithm(adjList)
    end = time.time()
    prim_time = (end - start) * 1000

    start = time.time()
    kruskal_weight = kruskalsAlgorithm(n, edgeList)
    end = time.time()
    kruskal_time = (end - start) * 1000

    return round(prim_time, 2), round(kruskal_time, 2), prim_weight, kruskal_weight

# ---------- TEST ----------
files = [
    ("10_sparseGraph.txt", "10 (Sparse)"),
    ("10_denseGraph.txt", "10 (Dense)"),
    ("100_sparseGraph.txt", "100 (Sparse)"),
    ("100_denseGraph.txt", "100 (Dense)"),
    ("1000_sparseGraph.txt", "1000 (Sparse)"),
    ("1000_denseGraph.txt", "1000 (Dense)"),
]

print("| N               | Prim Time (ms) | Kruskal Time (ms) | Prim MST | Kruskal MST |")
print("|-----------------|----------------|-------------------|----------|-------------|")

for file, label in files:
    try:
        pt, kt, pw, kw = runAndTimeAlgorithms(file)
        print(f"| {label:14}  | {pt:14} |{kt:18} |{pw:9} |{kw:12} |")
    except FileNotFoundError:
        print(f"| {label:14}  | FILE NOT FOUND")

| N               | Prim Time (ms) | Kruskal Time (ms) | Prim MST | Kruskal MST |
|-----------------|----------------|-------------------|----------|-------------|
| 10 (Sparse)     |            0.0 |               0.0 |      255 |         255 |
| 10 (Dense)      |            0.0 |               0.0 |      277 |         277 |
| 100 (Sparse)    |            0.0 |               1.0 |      inf |        2602 |
| 100 (Dense)     |           1.99 |              1.99 |      186 |         186 |
| 1000 (Sparse)   |           2.99 |              6.41 |      inf |       27683 |
| 1000 (Dense)    |         166.79 |             87.06 |     1000 |        1000 |
